# Fetching Data

In [91]:
# Import necessary libraries
import pandas as pd
import os
import json

In [ ]:
# YouTube Playlist Links to download
playlists_link = {
    "CoryKenshinRandomVideos": "https://www.youtube.com/playlist?list=PLxoHK1S7LhWSU230SAT7fDUsLiXCwaBMo",
    "MusicVideos": "https://youtube.com/playlist?list=PLBTanuC8SLeZyvv762fHIvS6jxdXsO4Ev&si=NsRFP84YsFtFItU4",
    "VinesComedyVideos" : "https://youtube.com/playlist?list=PLk37WUr698vmWZjWRWucJAxxuckNHJ6yq&si=vKMgZktUlBnfwpVK"
}
project_directory = "/mnt/d/Hackathon/DatathonRistek/Datathon_Techwiz" # Donovan Directory
os.chdir(project_directory)

In [93]:
# Download videos information from given playlists
def playlist_download(playlists: dict[str, str]) -> None:
    os.chdir(os.path.join(os.getcwd(), "VideoData"))
    for playlist_title, playlist_link in playlists.items():
        try:
            os.mkdir(f"{playlist_title}Playlist")
        except FileExistsError:
            continue
        os.chdir(os.path.join(os.getcwd(), f"{playlist_title}Playlist"))
        os.system(f"yt-dlp --write-info-json --skip-download --convert-thumbnails jpg --write-thumbnail -o \"%(playlist_index)s.%(ext)s\" {playlist_link}")
        os.chdir("..")

In [94]:
playlist_download(playlists_link)

In [107]:
# Convert .json files to pandas DataFrame
numerical_feature_json_keys = ["comment_count", "view_count", "like_count"]
text_feature_json_keys = ["title", "description", "channel"]
categories_feature_json_keys = ["tags", "categories"]
duration_string_feature_json_key = "duration_string"
playlist_json_file = "000.info.json"

def duration_string_to_seconds(duration_string: str) -> int:
    '''Converts duration strings to seconds'''
    multiplier = 1
    total_seconds = 0
    for digit in duration_string.split(":")[::-1]:
        total_seconds += int(digit) * multiplier
        multiplier *= 60
    return total_seconds

def video_json_to_dictionary(file_path: str) -> dict:
    '''Get specific features from .json file and return as dictionary pair of (name, value)'''
    with open(file_path, 'r') as f:
        result = dict()
        data = json.load(f)
        for key in numerical_feature_json_keys:
            result.update({key: data.get(key, 0)}) # get(key, default_value)
        for key in text_feature_json_keys:
            result.update({key: data.get(key, "")})
        for key in categories_feature_json_keys:
            result.update({key: " ".join(data.get(key, ""))})
        result.update({"duration": duration_string_to_seconds(data[duration_string_feature_json_key])})
    return result

def get_title_from_playlist_json(file_path: str) -> str:
    '''Opens .json info file for playlist, then returns the title'''
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data["title"]

def playlist_to_dataframes() -> dict[str, pd.DataFrame]:
    dataframes = dict()
    for root, dirs, files in os.walk(os.path.join(project_directory, "VideoData")):
        # Removes hidden directories from walk
        dirs[:] = [d for d in dirs if not d[0] == '.']
        # Skips directories with 0 files (this also skips playlist directories)
        if len(files) == 0: continue
        # Init playlist dataframe and title
        playlist_dataframe = pd.DataFrame()
        playlist_title = root.split("/")[::-1][0]       
        # Traverse files in subdirectory (playlist directory)
        for file in files:
            file_path = os.path.join(root, file) # absolute path of the file
            if file == playlist_json_file: # checks and sets playlist info .json
                continue
            elif file.endswith(".json"): # parse video .json
                video_index = [int(file.split(".")[0])]
                playlist_dataframe = pd.concat(
                    [
                        playlist_dataframe, pd.DataFrame(video_json_to_dictionary(file_path)
                        , index=video_index)
                    ]
                    , ignore_index=False # if not ignored, it uses the index of both dataframes
                )
        # Sorts rows alphabetically (for easier visualisation)
        playlist_dataframe = playlist_dataframe.sort_index()
        dataframes.update({playlist_title: playlist_dataframe})
    return dataframes

In [108]:
shit = playlist_to_dataframes()

In [114]:
cory_kenshin_vids_df = shit['CoryKenshinRandomVideosPlaylist']
music_vids_df = shit['MusicVideosPlaylist']
vine_vids_df = shit['VinesComedyVideosPlaylist']

cory_kenshin_vids_df.head()

,comment_count,view_count,like_count,title,description,channel,tags,categories,duration
1,295000,18270143,903187,Reacting to the Five Nights at Freddy's Movie ...,Trailer: https://www.youtube.com/watch?v=0VH9W...,CoryxKenshin,,Gaming,423
2,100000,3969031,449915,30,this video has no purpose other than for me to...,CoryxKenshin,30 coryxkenshin cory kenshin birthday 2022 rem...,Gaming,475
3,29000,4555735,373542,I Can't Hide This Any Longer,https://coryxkenshin.com GOGOGOGO BEFORE IT SE...,CoryxKenshin,i cant hide this any longer cant hide this any...,Gaming,937
4,130000,10211898,876158,Deleting My YouTube Channel.,Promise broken. Thank you for everything.,CoryxKenshin,deleting my youtube channel coryxkenshin cory ...,Gaming,1159
5,94000,10396076,1181262,YouTube: Racism and Favoritism,Never in a million years did I think I would u...,CoryxKenshin,youtube favoritism and racism coryxkenshin cor...,Gaming,838


In [115]:
music_vids_df.head()

,comment_count,view_count,like_count,title,description,channel,tags,categories,duration
1,4300000,8751645121,54956225,Luis Fonsi - Despacito ft. Daddy Yankee,“Despacito” disponible ya en todas las platafo...,Luis Fonsi,Luis Fonsi Despacito UMLE Latino Latin Pop Lui...,Music,282
2,2300000,6710469143,45186342,Wiz Khalifa - See You Again ft. Charlie Puth [...,Download the new Furious 7 Soundtrack Deluxe V...,Wiz Khalifa,Wiz Khalifa Blacc Hollywood Taylor Gang Atlant...,Music,237
3,1200000,6499341796,34496973,Ed Sheeran - Shape of You (Official Music Video),The official music video for Ed Sheeran - Shap...,Ed Sheeran,Ed Sheeran Shape Of You Ed Sheeran Shape Of Yo...,Music,263
4,435000,4252160992,16692077,Maroon 5 - Sugar (Official Music Video),Buy Sugar on iTunes: http://smarturl.it/M5V\n\...,Maroon 5,maroon sugar maroon she will be loved maroon o...,Music,301
5,524000,4191809528,18563457,OneRepublic - Counting Stars,Stream & Download OneRepublic’s latest album “...,OneRepublic,OneRepublic One Republic 1Republic Counting St...,Music,283


In [116]:
vine_vids_df.head()

,comment_count,view_count,like_count,title,description,channel,tags,categories,duration
1,12,19400,363,Funny Vines July 2024 (Part 1) TBT Clean Vine,New TBT Vine Videos with the Best Clean Vines ...,CooL Vines,Funny Vines Vine Vines best vines new vines ju...,Entertainment,1144
2,94,377724,5263,Funny Vines July 2022 (Part 2) TBT Clean Vine,New TBT Vine with the Best Clean Vines of July...,CooL Vines,Funny Vines Vine Vines best vines new vines Ju...,Entertainment,1265
3,80,242371,4564,Funny Vines April 2022 (Part 1) TBT Clean Vine,New TBT Vine with the Best Clean Vines of Apri...,CooL Vines,Funny Vines Vine Vines best vines new vines Ap...,Entertainment,1165
4,37,163481,2738,Funny Vines March 2022 (Part 1) TBT Clean Vine,New TBT Vine with the Best Clean Vines of Marc...,CooL Vines,Funny Vines Vine Vines best vines new vines Ma...,Entertainment,1261
5,77,212974,3639,Funny Vines March 2022 (Part 2) TBT Clean Vine,New TBT Vine with the Best Clean Vines of Marc...,CooL Vines,Funny Vines Vine Vines best vines new vines Ma...,Entertainment,1548


In [118]:
# Save the given data to CSV
os.chdir(os.path.join(project_directory, "CSVData"))
for title, dataframe in shit.items():
    dataframe.to_csv(f"{title}.csv")
os.chdir(project_directory)